<a href="https://colab.research.google.com/github/AtSourav/AE-w.-Bottleneck-Residual-Blocks/blob/main/AE_cifar10_residual_blocks_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For this notebook we shall use some custom python modules where we have defined some helper functions for plotting images, and more importantly some custom layers to implement residual blocks and bottleneck residual blocks.

The architecture here is a direct rephrasing of the architecture in the repo called L2-vs-L1-vs-CB-loss-in-deep-CNN-based-AE-on-cifar10
in terms of residual blocks.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow import math
import tensorflow_datasets as tfds
from keras import losses
from keras import layers
from keras import utils
from keras import metrics
from keras import backend as K
from keras import initializers

import os
import random
import matplotlib.pyplot as plt

# **Cloning the github repo with the custom modules**

In [2]:
! git clone https://github.com/AtSourav/AE-w.-Bottleneck-Residual-Blocks

%cd "/content/AE-w.-Bottleneck-Residual-Blocks"

import plotting_tools as plts
import Residual_blocks as resblock
import Bottleneck_residual_blocks as bresblock

Cloning into 'AE-w.-Bottleneck-Residual-Blocks'...
remote: Enumerating objects: 94, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 94 (delta 41), reused 56 (delta 17), pack-reused 0
Receiving objects: 100% (94/94), 41.50 KiB | 5.19 MiB/s, done.
Resolving deltas: 100% (41/41), done.
/content/AE-w.-Bottleneck-Residual-Blocks


# **Set-up, and dataset (cifar10) loading**

In [3]:
img_ht = 32
img_wd = 32

input_size = (img_ht,img_wd,3)
latent_dim = 256
batch_size = 128

initializer = initializers.HeNormal(seed=123)

In [4]:
ds = tfds.load('cifar10', split='train')
ds2= tfds.load('cifar10', split='test')

ds_to_np = list(ds.as_numpy_iterator())
ds2_to_np = list(ds2.as_numpy_iterator())

img_train = np.array([x['image'] for x in ds_to_np])
img_valid = np.array([x['image'] for x in ds2_to_np])

img_train = img_train/255
img_valid = img_valid/255

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incompleteW3QRTL/cifar10-train.tfrecord*...:   0%|          …

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incompleteW3QRTL/cifar10-test.tfrecord*...:   0%|          |…

Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.


# **Building the model**

In [9]:
encoder_input = keras.Input(shape=input_size)

x = resblock.bridge_residual_conv2D_block(64, 1, 3, initializer, 'min', use_bn = 'False')(encoder_input)

x = resblock.residual_conv2D_block(64, 1, 2, initializer, 'min', use_bn = 'False')(x)

#####################################################################################

x = layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding="valid")(x)

#####################################################################################

# the default padding is 'valid'

x = resblock.bridge_residual_conv2D_block(128, 2, 2, initializer, 'min', use_bn = 'False')(x)

x = resblock.residual_conv2D_block(128, 1, 1, initializer, 'min', use_bn = 'False')(x)

x = resblock.residual_conv2D_block(128, 2, 2, initializer, 'min', use_bn = 'False')(x)

x = resblock.residual_conv2D_block(128, 1, 1, initializer, 'min', use_bn = 'False')(x)

#####################################################################################

x = resblock.bridge_residual_conv2D_block(256, 2, 2, initializer, 'min', use_bn = 'False')(x)

x = resblock.residual_conv2D_block(256, 1, 1, initializer, 'min', use_bn = 'False')(x)

x = resblock.residual_conv2D_block(256, 2, 2, initializer, 'min', use_bn = 'False')(x)

x = resblock.residual_conv2D_block(256, 1, 1, initializer, 'min', use_bn = 'False')(x)

#####################################################################################

x = resblock.bridge_residual_conv2D_block(512, 2, 2, initializer, 'min', use_bn = 'False')(x)

x = resblock.residual_conv2D_block(512, 2, 2, initializer, 'min', use_bn = 'False')(x)

x = resblock.residual_conv2D_block(512, 1, 2, initializer, 'min', use_bn = 'False')(x)

x = resblock.residual_conv2D_block(512, 1, 1, initializer, 'min', use_bn = 'False')(x)

#####################################################################################

x = layers.Flatten()(x)

x = layers.Dense(3*latent_dim)(x)
x = layers.ReLU()(x)

x = layers.Dense(2*latent_dim)(x)
x = layers.ReLU()(x)


z = layers.Dense(latent_dim, name="z")(x)

encoder = keras.Model(encoder_input, z, name='encoder')
encoder.summary()

Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 bridge_residual_conv2d_blo  (None, 30, 30, 64)        2304      
 ck_4 (bridge_residual_conv                                      
 2D_block)                                                       
                                                                 
 residual_conv2d_block_10 (  (None, 29, 29, 64)        16704     
 residual_conv2D_block)                                          
                                                                 
 max_pooling2d_33 (MaxPooli  (None, 14, 14, 64)        0         
 ng2D)                                                           
                                                                 
 bridge_residual_conv2d_blo  (None, 12, 12, 128)       1073

In [10]:
latent_input = keras.Input(shape=(latent_dim,))

x = layers.Dense(2*latent_dim)(latent_input)
x = layers.ReLU()(x)

x = layers.Dense(3*latent_dim)(x)
x = layers.ReLU()(x)

x = layers.Dense(4*latent_dim)(x)
x = layers.ReLU()(x)

x = layers.Dense(2*2*1024)(x)
x = layers.ReLU()(x)

x = layers.Reshape((2,2,1024))(x)

x = resblock.bridge_residual_conv2Dtrans_block(1024, 1, 1, initializer, use_bn = 'False')(x)

##############################################################################################

x = resblock.bridge_residual_conv2Dtrans_block(512, 1, 1, initializer, use_bn = 'False')(x)

x = resblock.residual_conv2Dtrans_block(512, 2, 2, initializer, use_bn = 'False')(x)

#############################################################################################

x = resblock.bridge_residual_conv2Dtrans_block(256, 1, 1, initializer, use_bn = 'False')(x)

x = resblock.residual_conv2Dtrans_block(256, 2, 2, initializer, use_bn = 'False')(x)

x = resblock.residual_conv2Dtrans_block(256, 1, 2, initializer, use_bn = 'False')(x)

#############################################################################################

x = layers.UpSampling2D(size=(2, 2), data_format=None, interpolation='bilinear')(x)

#############################################################################################

x = resblock.bridge_residual_conv2Dtrans_block(128, 2, 2, initializer, use_bn = 'False')(x)

x = resblock.residual_conv2Dtrans_block(128, 2, 2, initializer, use_bn = 'False')(x)

############################################################################################

x = layers.UpSampling2D(size=(2, 2), data_format=None, interpolation='bilinear')(x)

###########################################################################################

x = resblock.bridge_residual_conv2D_block(128, 2, 2, initializer, 'min', use_bn = 'False')(x)

x = resblock.residual_conv2D_block(128, 1, 1, initializer, 'min', use_bn = 'False')(x)

###########################################################################################

x = resblock.bridge_residual_conv2D_block(64, 2, 2, initializer, 'min', use_bn = 'False')(x)

###########################################################################################

decoder_output = layers.Conv2D(3, 1, activation='sigmoid', padding='valid', kernel_initializer=initializer)(x)

decoder = keras.Model(latent_input, decoder_output, name="decoder")
decoder.summary()


Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 256)]             0         
                                                                 
 dense_8 (Dense)             (None, 512)               131584    
                                                                 
 re_lu_84 (ReLU)             (None, 512)               0         
                                                                 
 dense_9 (Dense)             (None, 768)               393984    
                                                                 
 re_lu_85 (ReLU)             (None, 768)               0         
                                                                 
 dense_10 (Dense)            (None, 1024)              787456    
                                                                 
 re_lu_86 (ReLU)             (None, 1024)              0   

In [11]:
decoder_out = decoder(encoder(encoder_input))
AE = keras.Model(encoder_input, decoder_out, name='AE')

AE.summary()

Model: "AE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 encoder (Functional)        (None, 256)               7422144   
                                                                 
 decoder (Functional)        (None, 32, 32, 3)         12404995  
                                                                 
Total params: 19827139 (75.63 MB)
Trainable params: 19812931 (75.58 MB)
Non-trainable params: 14208 (55.50 KB)
_________________________________________________________________


# **Training**

In [ ]:
K.clear_session()

In [12]:
#lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.0001, decay_steps=2000, decay_rate=0.7)
#callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
optim = keras.optimizers.Adam(learning_rate = 0.0001)
AE.compile(optimizer=optim,loss='mse')
history = AE.fit(img_train, img_train, batch_size=batch_size, validation_data=(img_valid, img_valid), epochs=75)

Epoch 1/75
391/391 [==============================] - 130s 271ms/step - loss: 0.0458 - val_loss: 0.0391
Epoch 2/75
391/391 [==============================] - 104s 265ms/step - loss: 0.0298 - val_loss: 0.0300
Epoch 3/75
391/391 [==============================] - 104s 265ms/step - loss: 0.0265 - val_loss: 0.0264
Epoch 4/75
391/391 [==============================] - 99s 252ms/step - loss: 0.0250 - val_loss: 0.0253
Epoch 5/75
391/391 [==============================] - 103s 265ms/step - loss: 0.0230 - val_loss: 0.0235
Epoch 6/75
391/391 [==============================] - 99s 252ms/step - loss: 0.0211 - val_loss: 0.0226
Epoch 7/75
391/391 [==============================] - 103s 265ms/step - loss: 0.0200 - val_loss: 0.0206
Epoch 8/75
391/391 [==============================] - 104s 265ms/step - loss: 0.0194 - val_loss: 0.0201
Epoch 9/75
391/391 [==============================] - 104s 265ms/step - loss: 0.0189 - val_loss: 0.0190
Epoch 10/75
391/391 [==============================] - 104s 265ms/